In [11]:
# Colab‑ready Python script: merge AQI & asthma data, plot interactively, and run a regression

# 0) If you haven’t already, install required packages:
# !pip install plotly statsmodels --quiet

import pandas as pd
import numpy as np
import plotly.express as px
import statsmodels.formula.api as smf

# 1) Load your CSVs (upload them into Colab’s file browser first)
aqi    = pd.read_csv('annual_aqi_by_county_2021.csv')
inc    = pd.read_csv('tableL6.csv')
state  = pd.read_csv('adult_asthma_by_state.csv')

# 2) Build a mapping from USPS codes → state names (for the income table)
usps = {
    'AL':'Alabama','AK':'Alaska','AZ':'Arizona','AR':'Arkansas','CA':'California','CO':'Colorado',
    'CT':'Connecticut','DE':'Delaware','FL':'Florida','GA':'Georgia','HI':'Hawaii','ID':'Idaho',
    'IL':'Illinois','IN':'Indiana','IA':'Iowa','KS':'Kansas','KY':'Kentucky','LA':'Louisiana',
    'ME':'Maine','MD':'Maryland','MA':'Massachusetts','MI':'Michigan','MN':'Minnesota',
    'MS':'Mississippi','MO':'Missouri','MT':'Montana','NE':'Nebraska','NV':'Nevada',
    'NH':'New Hampshire','NJ':'New Jersey','NM':'New Mexico','NY':'New York',
    'NC':'North Carolina','ND':'North Dakota','OH':'Ohio','OK':'Oklahoma','OR':'Oregon',
    'PA':'Pennsylvania','RI':'Rhode Island','SC':'South Carolina','SD':'South Dakota',
    'TN':'Tennessee','TX':'Texas','UT':'Utah','VT':'Vermont','VA':'Virginia','WA':'Washington',
    'WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming'
}

# 3) Prepare the income‑by‑asthma table:
inc = inc[['State','Weighted Numbere','Prevalence (Percent)']].copy()
inc.columns = ['StateCode','WeightedN','PrevPct']

# drop non‑numeric “WeightedN” rows (e.g. U.S. Total, Territories)
inc = inc[inc['WeightedN'].str.replace(',','').str.match(r'^\d+$')].copy()

# map state codes → full names
inc['State'] = inc['StateCode'].map(usps)
inc = inc.dropna(subset=['State'])

# convert types
inc['WeightedN'] = inc['WeightedN'].str.replace(',','').astype(int)
inc['PrevPct']   = inc['PrevPct'].astype(float)

# compute weighted state asthma prevalence
state_prev = (
    inc.groupby('State')
       .apply(lambda g: np.average(g['PrevPct'], weights=g['WeightedN']))
       .reset_index(name='AsthmaPrev')
)

# 4) Prepare the AQI table: average “Days with AQI” per state
aqi['State'] = aqi['State'].str.strip()
state_aqi = (
    aqi.groupby('State')['Days with AQI']
       .mean()
       .reset_index(name='AvgDaysWithAQI')
)

# 5) (Alternative) If you prefer CDC’s direct state‑prevalence instead of income‑weighted:
# state = state.rename(columns={'State or Territory':'State'})
# state['AsthmaPrev'] = state['Percent With Current Asthma (SE)'] \
#                         .str.extract(r'([\d\.]+)').astype(float)
# state_prev = state[['State','AsthmaPrev']]

# 6) Merge prevalence + AQI into one DataFrame
df = pd.merge(state_prev, state_aqi, on='State', how='inner')

# 7) Preview
print(df.head())

# 8) Interactive scatter: asthma vs AQI
fig = px.scatter(
    df,
    x='AvgDaysWithAQI',
    y='AsthmaPrev',
    size='AvgDaysWithAQI',
    color='State',
    hover_data={
        'State': True,
        'AvgDaysWithAQI': ':.1f',
        'AsthmaPrev': ':.2f'
    },
    title='Asthma Prevalence vs. Avg Days with AQI by State (2021)'
)
fig.update_layout(
    xaxis_title='Average Days with AQI',
    yaxis_title='Asthma Prevalence (%)',
    legend_title=False
)
fig.show()

# 9) Simple linear regression: test AQ effect on asthma
model = smf.ols('AsthmaPrev ~ AvgDaysWithAQI', data=df).fit()
print(model.summary())


<ipython-input-11-d27baee4197c>:48: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: np.average(g['PrevPct'], weights=g['WeightedN']))


        State  AsthmaPrev  AvgDaysWithAQI
0     Alabama   15.949241      294.800000
1      Alaska   14.152793      269.625000
2     Arizona   15.626098      361.153846
3    Arkansas   14.358813      250.363636
4  California   14.843011      358.924528


                            OLS Regression Results                            
Dep. Variable:             AsthmaPrev   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.022
Method:                 Least Squares   F-statistic:                 1.487e-05
Date:                Wed, 07 May 2025   Prob (F-statistic):              0.997
Time:                        12:56:35   Log-Likelihood:                -86.938
No. Observations:                  47   AIC:                             177.9
Df Residuals:                      45   BIC:                             181.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         15.5162      2.527      6.

In [27]:
# 0) (Optional) install required libs – run once in Colab
#!pip install plotly statsmodels --quiet

import pandas as pd
import numpy as np
import plotly.express as px
import statsmodels.formula.api as smf

# 1) Upload into Colab’s Files:
#    - annual_aqi_by_county_2021.csv
#    - tableL6.csv
#    - adult_asthma_by_state.csv

# 2) Load the CSVs
aqi   = pd.read_csv('annual_aqi_by_county_2021.csv')
inc   = pd.read_csv('tableL6.csv')
state = pd.read_csv('adult_asthma_by_state.csv')

# 3) Map USPS codes → full state names (for the income table)
usps = {
    'AL':'Alabama','AK':'Alaska','AZ':'Arizona','AR':'Arkansas','CA':'California','CO':'Colorado',
    'CT':'Connecticut','DE':'Delaware','FL':'Florida','GA':'Georgia','HI':'Hawaii','ID':'Idaho',
    'IL':'Illinois','IN':'Indiana','IA':'Iowa','KS':'Kansas','KY':'Kentucky','LA':'Louisiana',
    'ME':'Maine','MD':'Maryland','MA':'Massachusetts','MI':'Michigan','MN':'Minnesota',
    'MS':'Mississippi','MO':'Missouri','MT':'Montana','NE':'Nebraska','NV':'Nevada',
    'NH':'New Hampshire','NJ':'New Jersey','NM':'New Mexico','NY':'New York',
    'NC':'North Carolina','ND':'North Dakota','OH':'Ohio','OK':'Oklahoma','OR':'Oregon',
    'PA':'Pennsylvania','RI':'Rhode Island','SC':'South Carolina','SD':'South Dakota',
    'TN':'Tennessee','TX':'Texas','UT':'Utah','VT':'Vermont','VA':'Virginia','WA':'Washington',
    'WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming'
}

# 4) Build weighted asthma prevalence from the income table
inc = inc[['State','Weighted Numbere','Prevalence (Percent)']].copy()
inc.columns = ['StateCode','WeightedN','PrevPct']

# keep only numeric populations (drop “U.S. Total” & “Territories”)
inc = inc[inc['WeightedN'].str.replace(',','').str.match(r'^\d+$')].copy()

# map codes to names
inc['State'] = inc['StateCode'].map(usps)
inc = inc.dropna(subset=['State'])

# convert types
inc['WeightedN'] = inc['WeightedN'].str.replace(',','').astype(int)
inc['PrevPct']   = inc['PrevPct'].astype(float)

# compute state‐level weighted prevalence
state_prev = (
    inc
    .groupby('State')
    .apply(lambda g: np.average(g['PrevPct'], weights=g['WeightedN']))
    .reset_index(name='AsthmaPrev')
)

# 5) Aggregate county AQI → state average “Days with AQI”
aqi['State'] = aqi['State'].str.strip()
state_aqi = (
    aqi
    .groupby('State')['Days with AQI']
    .mean()
    .reset_index(name='AvgDaysWithAQI')
)

# 6) Merge into one DataFrame
df = pd.merge(state_prev, state_aqi, on='State', how='inner')

# 7) Plot scatter with OLS regression line
fig = px.scatter(
    df,
    x='AvgDaysWithAQI',
    y='AsthmaPrev',
    trendline='ols',
    trendline_color_override='red',
    hover_data=['State','AvgDaysWithAQI','AsthmaPrev'],
    title='Asthma Prevalence vs. Avg Days with AQI (2021)'
)
fig.update_layout(
    xaxis_title='Average Days with AQI',
    yaxis_title='Asthma Prevalence (%)',
    showlegend=False
)
fig.show()

# 8) Print the fitted line equation
res = px.get_trendline_results(fig).px_fit_results.iloc[0]
print(f"Fitted line:\n  AsthmaPrev = {res.params['Intercept']:.4f} + {res.params['AvgDaysWithAQI']:.6f} × AvgDaysWithAQI")


<ipython-input-27-5e4e7465654b>:52: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [28]:
# ─────────── install (first run) ───────────
# !pip install plotly statsmodels --quiet

import pandas as pd, numpy as np, plotly.express as px
import statsmodels.formula.api as smf

# 1) Load
aqi = pd.read_csv('annual_aqi_by_county_2021.csv')
inc = pd.read_csv('tableL6.csv')

# 2) Clean income‐asthma table
inc_clean = inc[['State','Income','Weighted Numbere','Prevalence (Percent)']].copy()
inc_clean.columns = ['StateCode','IncomeBracket','WeightedN','PrevPct']

# drop totals & territories
inc_clean = inc_clean[inc_clean['WeightedN'].str.replace(',','').str.match(r'^\d+$')]
inc_clean = inc_clean[inc_clean['IncomeBracket']!='Territories']

# map state codes→names
usps = {
    'AL':'Alabama','AK':'Alaska','AZ':'Arizona','AR':'Arkansas','CA':'California','CO':'Colorado',
    'CT':'Connecticut','DE':'Delaware','FL':'Florida','GA':'Georgia','HI':'Hawaii','ID':'Idaho',
    'IL':'Illinois','IN':'Indiana','IA':'Iowa','KS':'Kansas','KY':'Kentucky','LA':'Louisiana',
    'ME':'Maine','MD':'Maryland','MA':'Massachusetts','MI':'Michigan','MN':'Minnesota',
    'MS':'Mississippi','MO':'Missouri','MT':'Montana','NE':'Nebraska','NV':'Nevada',
    'NH':'New Hampshire','NJ':'New Jersey','NM':'New Mexico','NY':'New York',
    'NC':'North Carolina','ND':'North Dakota','OH':'Ohio','OK':'Oklahoma','OR':'Oregon',
    'PA':'Pennsylvania','RI':'Rhode Island','SC':'South Carolina','SD':'South Dakota',
    'TN':'Tennessee','TX':'Texas','UT':'Utah','VT':'Vermont','VA':'Virginia','WA':'Washington',
    'WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming'
}
inc_clean['State'] = inc_clean['StateCode'].map(usps)

# convert types
inc_clean['WeightedN'] = inc_clean['WeightedN'].str.replace(',','').astype(int)
inc_clean['PrevPct']   = inc_clean['PrevPct'].astype(float)

# 3) Exact bracket → midpoint map
income_map = {
    '< $15,000':             7500,
    '$15,000–<$25,000':     20000,
    '$25,000–<$50,000':     37500,
    '$50,000–<$75,000':     62500,
    '>=$75,000':            87500
}
inc_clean['IncomeMid'] = inc_clean['IncomeBracket'].map(income_map)

# 4) Weighted state‑level metrics
state_prev = (
    inc_clean.groupby('State')
             .apply(lambda g: np.average(g['PrevPct'], weights=g['WeightedN']))
             .reset_index(name='AsthmaPrev')
)
state_inc  = (
    inc_clean.groupby('State')
             .apply(lambda g: np.average(g['IncomeMid'], weights=g['WeightedN']))
             .reset_index(name='AvgIncome')
)

# 5) Aggregate AQI days per state
aqi['State'] = aqi['State'].str.strip()
state_aqi = aqi.groupby('State')['Days with AQI'].mean().reset_index(name='AvgDaysWithAQI')

# 6) Merge & regress
df = state_prev.merge(state_inc, on='State').merge(state_aqi, on='State')
model = smf.ols('AsthmaPrev ~ AvgDaysWithAQI + AvgIncome', data=df).fit()
print(model.summary())

# 7) Plot results
# (a) Asthma vs Income
px.scatter(df, x='AvgIncome', y='AsthmaPrev',
           trendline='ols', title='Asthma vs Income').show()

# (b) Asthma vs AQI coloured by Income
px.scatter(df, x='AvgDaysWithAQI', y='AsthmaPrev', color='AvgIncome',
           title='Asthma vs AQI (colour=Income)').show()


                            OLS Regression Results                            
Dep. Variable:             AsthmaPrev   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                 -0.033
Method:                 Least Squares   F-statistic:                    0.2633
Date:                Wed, 07 May 2025   Prob (F-statistic):              0.770
Time:                        14:41:34   Log-Likelihood:                -86.659
No. Observations:                  47   AIC:                             179.3
Df Residuals:                      44   BIC:                             184.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         16.7653      3.068      5.

<ipython-input-28-4e9f5fb32c3c>:51: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

<ipython-input-28-4e9f5fb32c3c>:56: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [21]:
# 1) (First run) install deps
#!pip install plotly statsmodels openpyxl --quiet

# 2) Imports
import pandas as pd, numpy as np, plotly.express as px
import statsmodels.formula.api as smf

# 3) Load your files
aqi    = pd.read_csv('annual_aqi_by_county_2021.csv')
inc    = pd.read_csv('tableL6.csv')
carbon = pd.read_excel('table4_shorter.xlsx', sheet_name='Table 4')

# 4) Dynamically pick the 2021 column (handles int or str headers)
year_cols = [c for c in carbon.columns if str(c).startswith('2021')]
if not year_cols:
    raise KeyError("No column starting with '2021' found in carbon dataset")
col2021 = year_cols[0]

carbon = carbon[['State', col2021]].rename(columns={col2021: 'CarbonPerCapita2021'})
carbon['State']               = carbon['State'].str.strip()
carbon['CarbonPerCapita2021'] = pd.to_numeric(carbon['CarbonPerCapita2021'], errors='coerce')

# 5) Build weighted asthma prevalence & average income
usps = {'AL':'Alabama','AK':'Alaska','AZ':'Arizona','AR':'Arkansas','CA':'California','CO':'Colorado',
        'CT':'Connecticut','DE':'Delaware','FL':'Florida','GA':'Georgia','HI':'Hawaii','ID':'Idaho',
        'IL':'Illinois','IN':'Indiana','IA':'Iowa','KS':'Kansas','KY':'Kentucky','LA':'Louisiana',
        'ME':'Maine','MD':'Maryland','MA':'Massachusetts','MI':'Michigan','MN':'Minnesota',
        'MS':'Mississippi','MO':'Missouri','MT':'Montana','NE':'Nebraska','NV':'Nevada',
        'NH':'New Hampshire','NJ':'New Jersey','NM':'New Mexico','NY':'New York',
        'NC':'North Carolina','ND':'North Dakota','OH':'Ohio','OK':'Oklahoma','OR':'Oregon',
        'PA':'Pennsylvania','RI':'Rhode Island','SC':'South Carolina','SD':'South Dakota',
        'TN':'Tennessee','TX':'Texas','UT':'Utah','VT':'Vermont','VA':'Virginia','WA':'Washington',
        'WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming'}

inc_clean = inc[['State','Income','Weighted Numbere','Prevalence (Percent)']].copy()
inc_clean.columns = ['StateCode','IncomeBracket','WeightedN','PrevPct']
inc_clean = inc_clean[
    inc_clean['WeightedN'].str.replace(',','').str.match(r'^\d+$') &
    (inc_clean['IncomeBracket']!='Territories')
]
inc_clean['State']     = inc_clean['StateCode'].map(usps)
inc_clean['WeightedN'] = inc_clean['WeightedN'].str.replace(',','').astype(int)
inc_clean['PrevPct']   = inc_clean['PrevPct'].astype(float)

income_map = {
    '< $15,000':            7500,
    '$15,000–<$25,000':    20000,
    '$25,000–<$50,000':    37500,
    '$50,000–<$75,000':    62500,
    '>=$75,000':           87500
}
inc_clean['IncomeMid'] = inc_clean['IncomeBracket'].map(income_map)

state_prev = (
    inc_clean.groupby('State')
             .apply(lambda g: np.average(g['PrevPct'], weights=g['WeightedN']))
             .reset_index(name='AsthmaPrev')
)
state_inc = (
    inc_clean.groupby('State')
             .apply(lambda g: np.average(g['IncomeMid'], weights=g['WeightedN']))
             .reset_index(name='AvgIncome')
)

# 6) Aggregate AQI
aqi['State'] = aqi['State'].str.strip()
state_aqi = aqi.groupby('State')['Days with AQI'] \
               .mean() \
               .reset_index(name='AvgDaysWithAQI')

# 7) Merge & regress
df = (state_prev
      .merge(state_inc, on='State')
      .merge(state_aqi, on='State')
      .merge(carbon,  on='State'))

model = smf.ols('AsthmaPrev ~ AvgDaysWithAQI + AvgIncome + CarbonPerCapita2021', data=df).fit()
print(model.summary())

# 8) Plot CO₂ effect
fig = px.scatter(
    df, x='CarbonPerCapita2021', y='AsthmaPrev',
    trendline='ols', trendline_color_override='green',
    hover_data=['State','CarbonPerCapita2021','AsthmaPrev'],
    title='Asthma Prevalence vs CO₂ Emissions Per Capita (2021)'
)
fig.update_layout(xaxis_title='CO₂ per Capita', yaxis_title='Asthma Prev (%)')
fig.show()


<ipython-input-21-81deb27d85cd>:56: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

<ipython-input-21-81deb27d85cd>:61: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



                            OLS Regression Results                            
Dep. Variable:             AsthmaPrev   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                 -0.042
Method:                 Least Squares   F-statistic:                    0.3814
Date:                Wed, 07 May 2025   Prob (F-statistic):              0.767
Time:                        14:25:37   Log-Likelihood:                -86.321
No. Observations:                  47   AIC:                             180.6
Df Residuals:                      43   BIC:                             188.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              17.5743    

In [31]:
# ─────── (1) Install & imports ───────
!pip install plotly statsmodels openpyxl --quiet

import pandas as pd, numpy as np, plotly.express as px
import statsmodels.formula.api as smf

# ─────── (2) Load all data ───────
aqi    = pd.read_csv('annual_aqi_by_county_2021.csv')
inc    = pd.read_csv('tableL6.csv')
carbon = pd.read_excel('table4_shorter.xlsx', sheet_name='Table 4')

# ─────── (3) Prepare carbon (2021) ───────
carbon = carbon[['State', 2021]].rename(columns={2021: 'Carbon2021'})
carbon['State'] = carbon['State'].str.strip()

# ─────── (4) Build asthma prevalence & income ───────
usps = {
  'AL':'Alabama','AK':'Alaska','AZ':'Arizona','AR':'Arkansas','CA':'California','CO':'Colorado',
  'CT':'Connecticut','DE':'Delaware','FL':'Florida','GA':'Georgia','HI':'Hawaii','ID':'Idaho',
  'IL':'Illinois','IN':'Indiana','IA':'Iowa','KS':'Kansas','KY':'Kentucky','LA':'Louisiana',
  'ME':'Maine','MD':'Maryland','MA':'Massachusetts','MI':'Michigan','MN':'Minnesota',
  'MS':'Mississippi','MO':'Missouri','MT':'Montana','NE':'Nebraska','NV':'Nevada',
  'NH':'New Hampshire','NJ':'New Jersey','NM':'New Mexico','NY':'New York',
  'NC':'North Carolina','ND':'North Dakota','OH':'Ohio','OK':'Oklahoma','OR':'Oregon',
  'PA':'Pennsylvania','RI':'Rhode Island','SC':'South Carolina','SD':'South Dakota',
  'TN':'Tennessee','TX':'Texas','UT':'Utah','VT':'Vermont','VA':'Virginia','WA':'Washington',
  'WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming'
}

inc_clean = inc[['State','Income','Weighted Numbere','Prevalence (Percent)']].copy()
inc_clean.columns = ['StateCode','IncomeBracket','WeightedN','PrevPct']
inc_clean = inc_clean[inc_clean['WeightedN'].str.replace(',','').str.match(r'^\d+$')]
inc_clean = inc_clean[inc_clean['IncomeBracket']!='Territories']
inc_clean['State']     = inc_clean['StateCode'].map(usps)
inc_clean['WeightedN'] = inc_clean['WeightedN'].str.replace(',','').astype(int)
inc_clean['PrevPct']   = inc_clean['PrevPct'].astype(float)

income_map = {
  '< $15,000':            7500,
  '$15,000–<$25,000':    20000,
  '$25,000–<$50,000':    37500,
  '$50,000–<$75,000':    62500,
  '>=$75,000':           87500
}
inc_clean['IncomeMid'] = inc_clean['IncomeBracket'].map(income_map)

state_prev = (
  inc_clean.groupby('State')
           .apply(lambda g: np.average(g['PrevPct'], weights=g['WeightedN']))
           .reset_index(name='AsthmaPrev')
)
state_inc = (
  inc_clean.groupby('State')
           .apply(lambda g: np.average(g['IncomeMid'], weights=g['WeightedN']))
           .reset_index(name='AvgIncome')
)

# ─────── (5) Aggregate pollutant‐“days” per state ───────
aqi['State'] = aqi['State'].str.strip()
state_poll = (
  aqi.groupby('State')
     .agg({
       'Days with AQI':'mean',
       'Days PM2.5':'mean',
       'Days Ozone':'mean',
       'Days NO2':'mean',
       'Days CO':'mean',
       'Days PM10':'mean'
     })
     .rename(columns={
       'Days with AQI':'AvgDaysWithAQI',
       'Days PM2.5':'AvgDaysPM25',
       'Days Ozone':'AvgDaysOzone',
       'Days NO2':'AvgDaysNO2',
       'Days CO':'AvgDaysCO',
       'Days PM10':'AvgDaysPM10'
     })
     .reset_index()
)

# ─────── (6) Merge everything ───────
df_all = (state_prev
          .merge(state_inc, on='State')
          .merge(state_poll, on='State')
          .merge(carbon,    on='State'))

# ─────── (7) Multivariate regression ───────
formula = ('AsthmaPrev ~ AvgDaysWithAQI + AvgIncome + Carbon2021 + '
           'AvgDaysPM25 + AvgDaysOzone + AvgDaysNO2 + AvgDaysCO + AvgDaysPM10')
model = smf.ols(formula, data=df_all).fit()
print(model.summary())

# ─────── (8) Visualize one pollutant’s effect ───────
fig = px.scatter(
  df_all, x='AvgDaysPM25', y='AsthmaPrev',
  trendline='ols', title='Asthma Prev vs Avg Days PM₂.₅'
)
fig.update_layout(xaxis_title='Average Days PM₂.₅', yaxis_title='Asthma Prev (%)')
fig.show()


<ipython-input-31-c723a3dc0efe>:49: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

<ipython-input-31-c723a3dc0efe>:54: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



                            OLS Regression Results                            
Dep. Variable:             AsthmaPrev   R-squared:                       0.181
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     1.234
Date:                Wed, 07 May 2025   Prob (F-statistic):              0.308
Time:                        14:42:49   Log-Likelihood:                -82.237
No. Observations:                  47   AIC:                             180.5
Df Residuals:                      39   BIC:                             195.3
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         16.8230      3.277      5.

In [32]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 1) Compute VIFs for your pollutant & AQI days block
X = df_all[['AvgDaysWithAQI','AvgDaysPM25','AvgDaysOzone','AvgDaysNO2','AvgDaysCO','AvgDaysPM10']]
vifs = pd.Series(
    [variance_inflation_factor(X.values, i) for i in range(X.shape[1])],
    index=X.columns
)
print("VIFs:\n", vifs)

# 2) If VIFs > 10, do a PCA on X
from sklearn.decomposition import PCA
X_scaled = (X - X.mean()) / X.std()
pca = PCA(n_components=2).fit(X_scaled)
df_all['Pollution_PC1'] = pca.transform(X_scaled)[:,0]
df_all['Pollution_PC2'] = pca.transform(X_scaled)[:,1]

# 3) New regression with PCs instead of raw days
import statsmodels.formula.api as smf
model_pca = smf.ols(
    'AsthmaPrev ~ AvgIncome + Carbon2021 + Pollution_PC1 + Pollution_PC2',
    data=df_all
).fit()
print(model_pca.summary())


VIFs:
 AvgDaysWithAQI    inf
AvgDaysPM25       inf
AvgDaysOzone      inf
AvgDaysNO2        inf
AvgDaysCO         inf
AvgDaysPM10       inf
dtype: float64
                            OLS Regression Results                            
Dep. Variable:             AsthmaPrev   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                 -0.021
Method:                 Least Squares   F-statistic:                    0.7643
Date:                Wed, 07 May 2025   Prob (F-statistic):              0.554
Time:                        14:43:00   Log-Likelihood:                -85.287
No. Observations:                  47   AIC:                             180.6
Df Residuals:                      42   BIC:                             189.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t

/usr/local/lib/python3.11/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning:

divide by zero encountered in scalar divide

